# MOSFET

## Metal-Oxide-Semiconductor Field-Effect Transistor

The MOSFET is the fundamental building block of modern digital electronics. Understanding its operation is essential for circuit design and device engineering.

**Learning Objectives:**
- Understand MOSFET structure and operation
- Simulate transfer characteristics (Id-Vg)
- Simulate output characteristics (Id-Vd)
- Extract threshold voltage and transconductance

---

## 1. MOSFET Physics

### 1.1 Structure

```
     Gate (Metal/Poly)
     ================
     |   Gate Oxide  |
     ================
  Source    Channel    Drain
  (N+)    (Inversion)   (N+)
  ====     -------     ====
    |                    |
    |    P-substrate     |
    +--------------------+
           Body
```

### 1.2 Key Parameters

- **Threshold voltage** ($V_{th}$): Gate voltage to create inversion
  $$V_{th} = V_{FB} + 2\phi_F + \frac{\sqrt{2\epsilon_s q N_a (2\phi_F)}}{C_{ox}}$$

- **Drain current** (linear region, $V_{ds} < V_{gs} - V_{th}$):
  $$I_d = \mu_n C_{ox} \frac{W}{L} \left[(V_{gs} - V_{th})V_{ds} - \frac{V_{ds}^2}{2}\right]$$

- **Drain current** (saturation, $V_{ds} > V_{gs} - V_{th}$):
  $$I_d = \frac{1}{2} \mu_n C_{ox} \frac{W}{L} (V_{gs} - V_{th})^2$$

- **Transconductance**:
  $$g_m = \frac{\partial I_d}{\partial V_{gs}} = \mu_n C_{ox} \frac{W}{L} (V_{gs} - V_{th})$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_mosfet

# Physical constants
q = 1.6e-19
eps_0 = 8.85e-14  # F/cm
eps_si = 11.7 * eps_0
eps_ox = 3.9 * eps_0
kT = 0.0259  # eV at 300K
ni = 1.5e10  # intrinsic carrier concentration (cm^-3)

# MOSFET parameters
L = 0.1e-4    # Channel length (cm) = 100nm
W = 1e-4      # Channel width (cm) = 1um
tox = 5e-7    # Oxide thickness (cm) = 5nm
Na = 1e18     # Substrate doping (cm^-3)
mu_n = 400    # Electron mobility (cm^2/V-s)

# Calculated parameters
Cox = eps_ox / tox  # Oxide capacitance (F/cm^2)
phi_F = kT * np.log(Na / ni)  # Fermi potential

print("MOSFET Parameters:")
print("="*40)
print(f"Channel length: {L*1e4:.0f} nm")
print(f"Oxide thickness: {tox*1e7:.0f} nm")
print(f"Cox = {Cox*1e6:.2f} uF/cm^2")
print(f"phi_F = {phi_F:.3f} V")

---

## 2. Creating a MOSFET Simulation

Let's create an NMOS transistor and examine its characteristics.

In [ ]:
# Create NMOS transistor
sim_nmos = create_mosfet(
    # Geometry
    channel_length=0.1,         # 100nm gate length
    gate_oxide_thickness=0.005, # 5nm oxide
    junction_depth=0.02,        # 20nm junction depth
    device_width=0.125,         # Device width
    device_depth=0.068,         # Substrate depth
    
    # Mesh
    nx=51,
    ny=51,
    
    # Doping
    channel_doping=1e18,        # Channel doping (p-type for NMOS)
    substrate_doping=5e16,      # Substrate doping
    source_drain_doping=1e20,   # N+ S/D doping
    device_type='nmos',
    
    # Models
    temperature=300,
    bgn=True,                   # Band-gap narrowing
    carriers=1,                 # Electrons only (for speed)
    
    # Output
    log_iv=True,
    iv_file="nmos_idvg"
)

print("NMOS Transistor Created")
print("="*40)
print("Channel length: 100 nm")
print("Gate oxide: 5 nm")
print("Channel doping: 1e18 cm^-3")

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_nmos.generate_deck())

---

## 3. Transfer Characteristics (Id-Vg)

The transfer characteristic shows drain current vs gate voltage at fixed drain voltage.

In [ ]:
# Create MOSFET with transfer characteristic sweep
sim_idvg = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='nmos',
    
    # Enable I-V logging
    log_iv=True,
    iv_file="idvg",
    
    # Transfer characteristic: sweep Vgs at fixed Vds
    vgs_sweep=(0.0, 1.5, 0.05),  # Vgs: 0 to 1.5V
    vds=0.1                       # Vds = 0.1V (linear region)
)

print("Transfer Characteristic Simulation")
print("="*40)
print("Gate voltage sweep: 0V to 1.5V")
print("Drain voltage: 0.1V (linear region)")
print("\nTo run: result = sim_idvg.run()")

In [ ]:
# Theoretical Id-Vg (simplified model)
Vth = 0.4  # Threshold voltage (V)
Vgs = np.linspace(0, 1.5, 100)
Vds = 0.1  # Linear region

# Calculate drain current (linear region)
Id = np.zeros_like(Vgs)
for i, vgs in enumerate(Vgs):
    if vgs > Vth:
        # Linear region formula
        Id[i] = mu_n * Cox * (W/L) * ((vgs - Vth) * Vds - Vds**2/2)

plt.figure(figsize=(12, 5))

# Linear scale
plt.subplot(1, 2, 1)
plt.plot(Vgs, Id * 1e6, 'b-', linewidth=2)
plt.axvline(x=Vth, color='r', linestyle='--', label=f'Vth = {Vth}V')
plt.xlabel('Gate Voltage Vgs (V)', fontsize=12)
plt.ylabel('Drain Current Id (uA)', fontsize=12)
plt.title('Transfer Characteristic (Linear Scale)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.5)

# Log scale (subthreshold region)
plt.subplot(1, 2, 2)
# Add subthreshold current
Id_sub = 1e-9 * np.exp((Vgs - Vth) / (2.3 * kT))  # Simplified subthreshold
Id_total = np.maximum(Id, Id_sub)
plt.semilogy(Vgs, Id_total, 'b-', linewidth=2)
plt.axvline(x=Vth, color='r', linestyle='--', label=f'Vth = {Vth}V')
plt.xlabel('Gate Voltage Vgs (V)', fontsize=12)
plt.ylabel('Drain Current Id (A)', fontsize=12)
plt.title('Transfer Characteristic (Log Scale)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.5)
plt.ylim(1e-12, 1e-3)

# Add subthreshold slope annotation
plt.annotate('Subthreshold\nslope ~60-100 mV/dec', 
             xy=(0.2, 1e-10), fontsize=10)

plt.tight_layout()
plt.show()

---

## 4. Output Characteristics (Id-Vd)

The output characteristic shows drain current vs drain voltage at different gate voltages.

In [ ]:
# Create MOSFET with output characteristic sweep
sim_idvd = create_mosfet(
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    channel_doping=1e18,
    source_drain_doping=1e20,
    device_type='nmos',
    
    # Enable I-V logging
    log_iv=True,
    iv_file="idvd",
    
    # Output characteristic: sweep Vds at fixed Vgs
    vds_sweep=(0.0, 1.5, 0.05),  # Vds: 0 to 1.5V
    vgs=1.0                       # Vgs = 1.0V
)

print("Output Characteristic Simulation")
print("="*40)
print("Drain voltage sweep: 0V to 1.5V")
print("Gate voltage: 1.0V")

In [ ]:
# Theoretical Id-Vd curves
Vds = np.linspace(0, 1.5, 100)
Vgs_values = [0.6, 0.8, 1.0, 1.2, 1.4]
Vth = 0.4

plt.figure(figsize=(10, 6))

for Vgs in Vgs_values:
    Id = np.zeros_like(Vds)
    Vdsat = Vgs - Vth
    
    for i, vds in enumerate(Vds):
        if Vgs > Vth:
            if vds < Vdsat:
                # Linear region
                Id[i] = mu_n * Cox * (W/L) * ((Vgs - Vth) * vds - vds**2/2)
            else:
                # Saturation region
                Id[i] = 0.5 * mu_n * Cox * (W/L) * (Vgs - Vth)**2
    
    plt.plot(Vds, Id * 1e6, linewidth=2, label=f'Vgs = {Vgs}V')
    
    # Mark saturation point
    if Vgs > Vth:
        plt.axvline(x=Vdsat, color='gray', linestyle=':', alpha=0.3)

plt.xlabel('Drain Voltage Vds (V)', fontsize=12)
plt.ylabel('Drain Current Id (uA)', fontsize=12)
plt.title('NMOS Output Characteristics', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.5)
plt.ylim(0, None)

# Add region labels
plt.annotate('Linear', xy=(0.2, 50), fontsize=10)
plt.annotate('Saturation', xy=(1.0, 50), fontsize=10)

plt.tight_layout()
plt.show()

---

## 5. NMOS vs PMOS

Let's compare NMOS and PMOS transistors.

In [ ]:
# NMOS transistor
sim_nmos = create_mosfet(
    channel_length=0.1,
    channel_doping=1e18,
    device_type='nmos',
    log_iv=True,
    vgs_sweep=(0.0, 1.5, 0.05),
    vds=0.1
)

# PMOS transistor
sim_pmos = create_mosfet(
    channel_length=0.1,
    channel_doping=1e18,
    device_type='pmos',
    log_iv=True,
    vgs_sweep=(0.0, -1.5, -0.05),  # Negative voltages for PMOS
    vds=-0.1                        # Negative Vds for PMOS
)

print("NMOS vs PMOS Comparison")
print("="*50)
print("\nNMOS:")
print("  - N+ source/drain in P-substrate")
print("  - Electrons as carriers")
print("  - Positive Vgs turns ON")
print("\nPMOS:")
print("  - P+ source/drain in N-well")
print("  - Holes as carriers")
print("  - Negative Vgs turns ON")
print("  - ~2-3x lower mobility than NMOS")

---

## 6. Channel Length Effects

Let's explore how channel length affects transistor behavior.

In [ ]:
# Compare different channel lengths
channel_lengths = [0.5, 0.2, 0.1, 0.05]  # microns

print("Channel Length Comparison:")
print("="*50)

for L_um in channel_lengths:
    sim = create_mosfet(
        channel_length=L_um,
        channel_doping=1e18,
        device_type='nmos',
        log_iv=True,
        iv_file=f"idvg_L{int(L_um*1000)}nm",
        vgs_sweep=(0.0, 1.5, 0.05),
        vds=0.1
    )
    print(f"L = {L_um*1000:.0f} nm: simulation configured")

In [ ]:
# Theoretical comparison
Vgs = np.linspace(0, 1.5, 100)
Vth = 0.4
Vds = 0.1

plt.figure(figsize=(10, 6))

for L_um in channel_lengths:
    L_cm = L_um * 1e-4
    Id = np.zeros_like(Vgs)
    
    for i, vgs in enumerate(Vgs):
        if vgs > Vth:
            Id[i] = mu_n * Cox * (W/L_cm) * ((vgs - Vth) * Vds - Vds**2/2)
    
    plt.semilogy(Vgs, Id + 1e-12, linewidth=2, label=f'L = {L_um*1000:.0f} nm')

plt.xlabel('Gate Voltage Vgs (V)', fontsize=12)
plt.ylabel('Drain Current Id (A)', fontsize=12)
plt.title('Effect of Channel Length on Id-Vg', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.5)

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("  - Shorter channel = higher current (Id ~ 1/L)")
print("  - Short-channel effects appear at small L")
print("  - Vth may decrease (DIBL) at short L")

---

## 7. Threshold Voltage Extraction

Methods to extract Vth from simulation data.

In [ ]:
# Generate sample Id-Vg data
Vgs = np.linspace(0, 1.5, 100)
Vth_true = 0.4
Vds = 0.1
L_cm = 0.1e-4

# Calculate Id with some noise
Id = np.zeros_like(Vgs)
for i, vgs in enumerate(Vgs):
    if vgs > Vth_true:
        Id[i] = mu_n * Cox * (W/L_cm) * ((vgs - Vth_true) * Vds - Vds**2/2)

# Add subthreshold current
Id_sub = 1e-9 * np.exp((Vgs - Vth_true) / (2.3 * kT))
Id_total = np.maximum(Id, Id_sub)

# Method 1: Linear extrapolation
# Find max gm point
gm = np.gradient(Id_total, Vgs)
max_gm_idx = np.argmax(gm)
slope = gm[max_gm_idx]
intercept = Id_total[max_gm_idx] - slope * Vgs[max_gm_idx]
Vth_linear = -intercept / slope

# Method 2: Constant current method
I_th = 1e-7  # Threshold current criterion
Vth_const = Vgs[np.argmin(np.abs(Id_total - I_th))]

plt.figure(figsize=(12, 5))

# Linear extrapolation method
plt.subplot(1, 2, 1)
plt.plot(Vgs, Id_total * 1e6, 'b-', linewidth=2, label='Id-Vg')
Vgs_extrap = np.linspace(Vth_linear - 0.2, Vgs[max_gm_idx] + 0.2, 50)
Id_extrap = slope * Vgs_extrap + intercept
plt.plot(Vgs_extrap, Id_extrap * 1e6, 'r--', linewidth=2, label='Linear fit')
plt.axvline(x=Vth_linear, color='g', linestyle=':', label=f'Vth = {Vth_linear:.3f}V')
plt.xlabel('Vgs (V)', fontsize=12)
plt.ylabel('Id (uA)', fontsize=12)
plt.title('Linear Extrapolation Method', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.5)

# Constant current method
plt.subplot(1, 2, 2)
plt.semilogy(Vgs, Id_total, 'b-', linewidth=2, label='Id-Vg')
plt.axhline(y=I_th, color='r', linestyle='--', label=f'Ith = {I_th:.0e} A')
plt.axvline(x=Vth_const, color='g', linestyle=':', label=f'Vth = {Vth_const:.3f}V')
plt.xlabel('Vgs (V)', fontsize=12)
plt.ylabel('Id (A)', fontsize=12)
plt.title('Constant Current Method', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.5)

plt.tight_layout()
plt.show()

print(f"\nThreshold Voltage Extraction:")
print(f"  True Vth: {Vth_true:.3f} V")
print(f"  Linear extrapolation: {Vth_linear:.3f} V")
print(f"  Constant current: {Vth_const:.3f} V")

---

## 8. Complete Simulation Example

In [ ]:
# Complete MOSFET characterization
sim_complete = create_mosfet(
    # Geometry
    channel_length=0.1,
    gate_oxide_thickness=0.005,
    junction_depth=0.02,
    device_width=0.125,
    device_depth=0.068,
    
    # Mesh
    nx=61,
    ny=61,
    
    # Doping
    channel_doping=1e18,
    substrate_doping=5e16,
    source_drain_doping=1e20,
    device_type='nmos',
    
    # Models
    temperature=300,
    bgn=True,
    carriers=2,  # Both electrons and holes
    
    # Output
    log_iv=True,
    iv_file="mosfet_complete",
    
    # Transfer characteristic
    vgs_sweep=(0.0, 1.5, 0.02),
    vds=0.05
)

print("Complete MOSFET Simulation")
print("="*50)
print(sim_complete.generate_deck())

---

## 9. Exercises

### Exercise 1: Oxide Thickness Effect
Compare MOSFETs with different oxide thicknesses (2nm, 5nm, 10nm). How does this affect Vth and transconductance?

### Exercise 2: Doping Optimization
Find the optimal channel doping for minimum subthreshold swing while maintaining reasonable Vth.

### Exercise 3: DIBL Analysis
Measure the transfer characteristic at Vds = 0.05V and Vds = 1.0V. Calculate the DIBL (Drain-Induced Barrier Lowering).

In [ ]:
# Exercise 1: Oxide thickness comparison
tox_values = [0.002, 0.005, 0.010]  # 2nm, 5nm, 10nm

print("Oxide Thickness Comparison:")
print("="*50)

for tox in tox_values:
    Cox_calc = eps_ox / (tox * 1e-4)  # F/cm^2
    
    sim = create_mosfet(
        gate_oxide_thickness=tox,
        channel_doping=1e18,
        device_type='nmos',
        log_iv=True,
        iv_file=f"tox_{int(tox*1000)}nm",
        vgs_sweep=(0.0, 1.5, 0.05),
        vds=0.1
    )
    
    print(f"\ntox = {tox*1000:.0f} nm:")
    print(f"  Cox = {Cox_calc*1e6:.2f} uF/cm^2")
    print(f"  Higher Cox -> Higher gm -> Faster switching")

---

## Summary

In this notebook, you learned:

1. **MOSFET Structure**: Source, drain, gate, and channel regions
2. **Transfer Characteristics**: Id vs Vgs, threshold voltage extraction
3. **Output Characteristics**: Linear and saturation regions
4. **NMOS vs PMOS**: Different carrier types and polarities
5. **Scaling Effects**: Impact of channel length on performance

**Key Equations:**
- Linear region: $I_d = \mu C_{ox}\frac{W}{L}[(V_{gs}-V_{th})V_{ds} - \frac{V_{ds}^2}{2}]$
- Saturation: $I_d = \frac{1}{2}\mu C_{ox}\frac{W}{L}(V_{gs}-V_{th})^2$

**Next**: [05 - BJT](05_BJT.ipynb) - Bipolar Junction Transistors